In [1]:
from decouple import config

In [2]:
assert config('ALPHAVANTAGE_API_KEY', default=None, cast=str) is not None


In [3]:
ALPHAVANTAGE_API_KEY = config('ALPHAVANTAGE_API_KEY', default=None, cast=str)

In [4]:
import requests


params = {'function': 'TIME_SERIES_INTRADAY', 'ticker': 'AAPL', 'interval': '5min', 'apikey': ALPHAVANTAGE_API_KEY}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={apikey}'.format(**params)
r = requests.get(url)
data = r.json()


In [5]:
data

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2024-12-19 19:55:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2024-12-19 19:55:00': {'1. open': '248.8900',
   '2. high': '249.1200',
   '3. low': '248.6200',
   '4. close': '248.7900',
   '5. volume': '8563'},
  '2024-12-19 19:50:00': {'1. open': '249.1100',
   '2. high': '249.2000',
   '3. low': '248.6200',
   '4. close': '248.7206',
   '5. volume': '6536'},
  '2024-12-19 19:45:00': {'1. open': '249.0200',
   '2. high': '249.3000',
   '3. low': '249.0200',
   '4. close': '249.1000',
   '5. volume': '2785'},
  '2024-12-19 19:40:00': {'1. open': '249.2800',
   '2. high': '249.3900',
   '3. low': '249.0200',
   '4. close': '249.0200',
   '5. volume': '3141'},
  '2024-12-19 19:35:00': {'1. open': '249.2450',
   '2. high': '249.4900',
   '3. low': '249.0222',
   '4. close': '249

In [8]:

dataset_keys = [x for x in data.keys() if not x.startswith('Meta Data')]

dataset_keys


['Time Series (5min)']

In [10]:
results = data[dataset_keys[0]]

results


{'2024-12-19 19:55:00': {'1. open': '248.8900',
  '2. high': '249.1200',
  '3. low': '248.6200',
  '4. close': '248.7900',
  '5. volume': '8563'},
 '2024-12-19 19:50:00': {'1. open': '249.1100',
  '2. high': '249.2000',
  '3. low': '248.6200',
  '4. close': '248.7206',
  '5. volume': '6536'},
 '2024-12-19 19:45:00': {'1. open': '249.0200',
  '2. high': '249.3000',
  '3. low': '249.0200',
  '4. close': '249.1000',
  '5. volume': '2785'},
 '2024-12-19 19:40:00': {'1. open': '249.2800',
  '2. high': '249.3900',
  '3. low': '249.0200',
  '4. close': '249.0200',
  '5. volume': '3141'},
 '2024-12-19 19:35:00': {'1. open': '249.2450',
  '2. high': '249.4900',
  '3. low': '249.0222',
  '4. close': '249.2800',
  '5. volume': '3867'},
 '2024-12-19 19:30:00': {'1. open': '249.1000',
  '2. high': '249.7900',
  '3. low': '249.0000',
  '4. close': '249.2450',
  '5. volume': '874507'},
 '2024-12-19 19:25:00': {'1. open': '248.9000',
  '2. high': '249.2000',
  '3. low': '248.6900',
  '4. close': '249.

In [11]:
timestamp_str = list(results.keys())[0]

timestamp_str


'2024-12-19 19:55:00'

In [ ]:
timestamp_format = '%Y-%m-%d %H:%M:%S'
eastern_timezone = pytz.timezone('US/Eastern')
utc_timezone = pytz.utc

timestamp = eastern_timezone.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc_timezone)


In [12]:
results[timestamp_str]


{'1. open': '248.8900',
 '2. high': '249.1200',
 '3. low': '248.6200',
 '4. close': '248.7900',
 '5. volume': '8563'}

In [15]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_polygon_result(timestamp_str, result):
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    eastern_timezone = pytz.timezone('US/Eastern')
    utc_timezone = pytz.utc

    timestamp = eastern_timezone.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc_timezone)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),        
        "number_of_transactions": None,
        'volume': int(result['5. volume']),
        "volume_weighted_average": None,
        "time": timestamp
    }

transform_polygon_result(timestamp_str, results[timestamp_str])


{'open_price': Decimal('248.8900'),
 'close_price': Decimal('248.7900'),
 'high_price': Decimal('249.1200'),
 'low_price': Decimal('248.6200'),
 'number_of_transactions': None,
 'volume': 8563,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 12, 20, 0, 55, tzinfo=<UTC>)}

In [16]:
for timestamp_str, result in results.items():
    print(transform_polygon_result(timestamp_str, result))


{'open_price': Decimal('248.8900'), 'close_price': Decimal('248.7900'), 'high_price': Decimal('249.1200'), 'low_price': Decimal('248.6200'), 'number_of_transactions': None, 'volume': 8563, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 20, 0, 55, tzinfo=<UTC>)}
{'open_price': Decimal('249.1100'), 'close_price': Decimal('248.7206'), 'high_price': Decimal('249.2000'), 'low_price': Decimal('248.6200'), 'number_of_transactions': None, 'volume': 6536, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 20, 0, 50, tzinfo=<UTC>)}
{'open_price': Decimal('249.0200'), 'close_price': Decimal('249.1000'), 'high_price': Decimal('249.3000'), 'low_price': Decimal('249.0200'), 'number_of_transactions': None, 'volume': 2785, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 12, 20, 0, 45, tzinfo=<UTC>)}
{'open_price': Decimal('249.2800'), 'close_price': Decimal('249.0200'), 'high_price': Decimal('249.3900'), 'low_price': Decimal('249.0200'), 'number